# MDF data analysis

In [1]:
from load_data import train, test, BIG, TRAIN, TEST
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

TRAIN (8564, 42)
TEST  (3671, 42)
BIG  (12235, 43)


/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from visualization_helper import display_groups, display_unique_count, display_unique_col_values, split_series_values
from preprocessing_helper import get_unique_noconst

In [3]:
print train.shape, train.columns.values
print test.shape, test.columns.values

(8564, 42) [u'libelle' u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon'
 u'libelle_tube' u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier'
 u'libelle_sachet' u'libelle_comprime' u'libelle_gelule' u'libelle_film'
 u'libelle_poche' u'libelle_capsule' u'nb_plaquette' u'nb_ampoule'
 u'nb_flacon' u'nb_tube' u'nb_stylo' u'nb_seringue' u'nb_pilulier'
 u'nb_sachet' u'nb_comprime' u'nb_gelule' u'nb_film' u'nb_poche'
 u'nb_capsule' u'nb_ml' u'statut' u'etat commerc' u'agrement col'
 u'tx rembours' u'forme pharma' u'voies admin' u'statut admin'
 u'date declar annee' u'date amm annee' u'type proc' u'titulaires'
 u'substances' u'prix' 'source']
(3671, 42) [u'id' u'libelle' u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon'
 u'libelle_tube' u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier'
 u'libelle_sachet' u'libelle_comprime' u'libelle_gelule' u'libelle_film'
 u'libelle_poche' u'libelle_capsule' u'nb_plaquette' u'nb_ampoule'
 u'nb_flacon' u'nb_tube' u'nb_stylo' u'nb_s

Remove some useless columns

In [4]:
input_df = train.drop(['prix','libelle'], axis=1)
# input_df = test.drop(['libelle'], axis=1)

In [5]:
df__noid_noprix = input_df
print df__noid_noprix.shape, df__noid_noprix.columns.values

(8564, 40) [u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon' u'libelle_tube'
 u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier' u'libelle_sachet'
 u'libelle_comprime' u'libelle_gelule' u'libelle_film' u'libelle_poche'
 u'libelle_capsule' u'nb_plaquette' u'nb_ampoule' u'nb_flacon' u'nb_tube'
 u'nb_stylo' u'nb_seringue' u'nb_pilulier' u'nb_sachet' u'nb_comprime'
 u'nb_gelule' u'nb_film' u'nb_poche' u'nb_capsule' u'nb_ml' u'statut'
 u'etat commerc' u'agrement col' u'tx rembours' u'forme pharma'
 u'voies admin' u'statut admin' u'date declar annee' u'date amm annee'
 u'type proc' u'titulaires' u'substances' 'source']


Check for dupicate rows and lets look at the columns with only one unique value.

In [6]:
df__noid_noprix_unique_noconst = get_unique_noconst(df__noid_noprix, verbose=True)

-- get_unique_noconst --
- Number of found duplicated rows :  1164
- Remove constant columns :  (7400, 40) -> (7400, 39)
- Dropped const columns :  ['source']


Columns description : 

In [7]:
type_groups_func = lambda df : df.columns.to_series().groupby(df.dtypes)
type_groups = type_groups_func(df__noid_noprix_unique_noconst)

In [8]:
display_groups(type_groups)


-- object --

	statut
	etat commerc
	agrement col
	tx rembours
	forme pharma
	voies admin
	statut admin
	type proc
	titulaires
	substances

-- int64 --

	libelle_plaquette
	libelle_ampoule
	libelle_flacon
	libelle_tube
	libelle_stylo
	libelle_seringue
	libelle_pilulier
	libelle_sachet
	libelle_comprime
	libelle_gelule
	libelle_film
	libelle_poche
	libelle_capsule
	nb_plaquette
	nb_ampoule
	nb_flacon
	nb_tube
	nb_stylo
	nb_seringue
	nb_pilulier
	nb_sachet
	nb_comprime
	nb_gelule
	nb_film
	nb_poche
	nb_capsule
	date declar annee
	date amm annee

-- float64 --

	nb_ml


Identify and separate the numeric and non numeric rows.
Report NaN values on numerical columns

In [ ]:
numeric_cols = []
nonnum_cols = []
for key in type_groups.groups:
    if np.issubdtype(key, np.number):
        numeric_cols.extend(type_groups.get_group(key).values)
    else:
        nonnum_cols.extend(type_groups.get_group(key).values)
        
nb_isnan = np.sum(df__noid_noprix_unique_noconst[numeric_cols].apply(np.isnan))

In [ ]:
print nb_isnan

Identify non numeric rows.

In [ ]:
df__noid_noprix_unique_noconst[nonnum_cols].tail(5)

In [ ]:
unique_col_values = display_unique_col_values(df__noid_noprix_unique_noconst[nonnum_cols])
unique_col_values

In [ ]:
index = 5
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df

In [ ]:
index = 8
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df

In [ ]:
index = 9
ll = unique_col_values.iloc[index][1]
res = split_series_values(unique_col_values.iloc[index][2:ll])    
df = display_unique_count(res)
print np.sum(df['Count'])
print df

Encode categorical features

In [ ]:
df = df__noid_noprix_unique_noconst[nonnum_cols].drop(['substances', 'voies admin'], axis=1)
out_df = pd.DataFrame(columns=df.columns, index=df.index)
for c in df.columns:
    le = LabelEncoder()    
    le.fit(df[c])
    out_df[c] = le.transform(df[c])
out_df